## Aim of This notebook 

The aim of this notebook is to create two shape file from the tables that we have already in our database.

In [1]:
import pandas as pd
import psycopg2
import credential as  creds 

## Connect to the Database

In [2]:
# connect to our credentioal file
conn_string = "host=" + creds.PGHOST + " port=" + creds.PORT + " dbname=" + creds.PGDATABASE + " user=" + creds.PGUSER \
    + " password=" + creds.PGPASSWORD


In [3]:
conn = psycopg2.connect(conn_string)

## Read the table from database

In this step, we need to read the messstelle table which is already in our database to the pandas dataframe.

In [4]:
quary = """ select * from Openhype.messstelle """

In [5]:
df = pd.read_sql(quary, conn,)

## Drop the geom column

The table has "geom" column which consist of the geometry information from e32 and n32 columns but we need to delete this column and create this column with another format which can be transfered to geopandas datafarme.   


In [6]:
df.drop('geom', axis = 1, inplace = True)

## Essentioanl packages which need for geopandas dataframe 

In [7]:
import geopandas as gpd
from shapely.geometry import Point

## Create geodata frame

with the helpf of geopandas we can create geo pandas dataframe. in here simply we can create the geometry column again but with another method.

In [8]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.e32, df.n32), crs="EPSG:25832")

In [9]:
gdf

,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gwhorizont_id,gwhorizont,...,eigentuemer,betreiber,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm,geometry
0,55754,279581312,Birkhof,349787,5628911,1.0,oeffentlich,05362012,None,None,...,None,None,200.0,100.0,65.0,NaN,1100.0,14231.0,14031.0,POINT (349787.000 5628911.000)
1,50232,285004438,Meinweg,302480,5675217,1.0,privat,05166020,6B,"Zwischenmittel, sandig",...,None,None,400.0,400.0,50.0,NaN,7000.0,660.0,260.0,POINT (302480.000 5675217.000)
2,51401,285008717,Schloï¿½ Wickrath 4 a,319117,5667318,1.0,oeffentlich,05116000,16,Jüngere Hauptterrassen mit Lößauflagerung,...,None,None,100.0,NaN,40.0,NaN,283.0,5749.0,5649.0,POINT (319117.000 5667318.000)
3,53063,285011613,Genholland,315126,5667547,1.0,privat,05116000,16,Jüngere Hauptterrassen mit Lößauflagerung,...,None,None,200.0,200.0,50.0,NaN,3200.0,4687.0,4487.0,POINT (315126.000 5667547.000)
4,63308,215776926,Weiler in der Ebene,338347,5621898,NaN,oeffentlich,05366044,7A,Sande und Tone,...,None,None,200.0,400.0,50.0,NaN,9600.0,6262.0,6062.0,POINT (338347.000 5621898.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71114,42859,279577813,Hochkirchen,356650,5638850,1.0,privat,05315000,4,"Zwischenmittel, sandig",...,Erftverband Raum 5 ...,Erftverband Raum 5 ...,10.0,0.0,50.0,NaN,4425.0,739.0,729.0,POINT (356650.000 5638850.000)
71115,53576,104462152,DVA H1 DEP VARL,470650,5806250,1.0,privat,05770040,None,None,...,Altdeponie Varl ...,Altdeponie Varl ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (470650.000 5806250.000)
71116,53577,104462164,DVA H2 DEP VARL,471050,5806250,1.0,privat,05770040,None,None,...,Altdeponie Varl ...,Altdeponie Varl ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (471050.000 5806250.000)
71117,42901,279579810,Bruehl 1,353599,5633201,1.0,oeffentlich,05362012,18,Untere- und mittlere Mittelterrassen mit Lößau...,...,None,None,400.0,100.0,50.0,NaN,2600.0,4028.0,3628.0,POINT (353599.000 5633201.000)


## Create the path of output

In [11]:
data_in_dir = r"../data/shape_file/"

file_fname = r"messstelle_local.gpkg"

file_pfname = data_in_dir + file_fname

## Create the shape file in the above path

In [12]:


gdf.to_file(file_pfname, layer='messstelle_local', driver="GPKG")

## Second shape file.
Now we want to create the second shape file.

* This shape file is a little bit complex than the previous one

## Read the table from Database

We don't want all the rows in messwert table, we want only nitrate and the nitrate stoff number is 1244

In [27]:
quary2 = """ select * from Openhype.messwert where stoff_nr = '1244' """

In [28]:
df1 = pd.read_sql(quary2, conn,)

In [29]:
df1

,sl_nr,messstelle_id,pna_id,datum_pn,stoff_nr,probengut,messergebnis_c,messergebnis_hinweis,bestimmungsgrenze,masseinheit,trennverfahren,verfahren,vor_ort,herkunft,aktual_dat,erstell_dat,messergebnis_cm
0,13215708,91161356,0/2019/90138,2019-02-27,1244,Grundwasser,26.1200,None,0.05,mg/l,Gesamtgehalt,DIN ISO 15923-1 - DI - D49 - 1,,LIMS_LANUV,2019-04-11,2019-04-11,=
1,13215766,91164916,0/2019/90149,2019-02-27,1244,Grundwasser,1.9900,None,0.05,mg/l,Gesamtgehalt,DIN ISO 15923-1 - DI - D49 - 1,,LIMS_LANUV,2019-04-11,2019-04-11,=
2,12444960,20103062,0/2018/91183,2018-12-17,1244,Grundwasser,5.3100,None,1.33,mg/l,Gesamtgehalt,DIN ISO 15923-1 - DI - D49 - 1,,LIMS_LANUV,2019-03-15,2019-02-16,=
3,13190542,46511283,3/1995/2,1995-10-11,1244,Grundwasser,7.0880,None,NaN,mg/l,Gesamtgehalt,nach Laborjournal,,HYGRISC_3,2019-03-20,2019-03-20,=
4,13190629,46511283,3/1996/3,1996-09-25,1244,Grundwasser,5.4932,None,NaN,mg/l,Gesamtgehalt,nach Laborjournal,,HYGRISC_3,2019-03-20,2019-03-20,=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73894,17442835,70203910,0/2020/2,2020-03-16,1244,Grundwasser,10.0000,None,NaN,mg/l,Gesamtgehalt,nach Laborjournal,,HYGRISC_1,2021-05-15,2021-05-15,=
73895,17443741,279489511,0/2020/1,2020-03-26,1244,Grundwasser,136.0000,None,NaN,mg/l,Gesamtgehalt,nach Laborjournal,,HYGRISC_1,2021-05-15,2021-05-15,=
73896,17457118,26500139,0/2021/1,2021-02-24,1244,Grundwasser,19.7000,None,NaN,mg/l,Gesamtgehalt,DIN ISO 15923-1 - DI - D49 - 1,,LIMS_LANUV,2021-05-18,2021-05-18,=
73897,17456990,26500115,0/2021/1,2021-02-24,1244,Grundwasser,18.4600,None,NaN,mg/l,Gesamtgehalt,DIN ISO 15923-1 - DI - D49 - 1,,LIMS_LANUV,2021-05-18,2021-05-18,=


## Merge the data

In [30]:
df1_n = df1[[ 'messstelle_id', 'datum_pn', 'stoff_nr', 'messergebnis_c', 'messergebnis_hinweis', 'messergebnis_cm']]

In [31]:
gdf_n = gdf[['messstelle_id', 'name', 'gemeinde_id', 'geometry' ]]

In [32]:
df_merg = pd.merge(df1_n, gdf_n, on = ['messstelle_id'])

In [33]:
df_merg

,messstelle_id,datum_pn,stoff_nr,messergebnis_c,messergebnis_hinweis,messergebnis_cm,name,gemeinde_id,geometry
0,91161356,2019-02-27,1244,26.12000,None,=,Grasweg Eikeloh,05974016,POINT (457990.000 5719038.000)
1,91161356,2006-09-07,1244,25.58806,None,=,Grasweg Eikeloh,05974016,POINT (457990.000 5719038.000)
2,91161356,2010-06-28,1244,26.38492,None,=,Grasweg Eikeloh,05974016,POINT (457990.000 5719038.000)
3,91161356,2014-08-11,1244,24.61412,None,=,Grasweg Eikeloh,05974016,POINT (457990.000 5719038.000)
4,91161356,2017-05-02,1244,25.23390,None,=,Grasweg Eikeloh,05974016,POINT (457990.000 5719038.000)
...,...,...,...,...,...,...,...,...,...
73894,110039890,2020-08-06,1244,66.00000,None,=,LGD Albersloh 01,05570040,POINT (411437.000 5747233.000)
73895,60000211,2020-08-04,1244,1.00000,Konzentration zu gering zur Bestimmung,<,LGD Vreden 01,05554068,POINT (349735.000 5770649.000)
73896,60070020,2020-08-05,1244,71.00000,None,=,LGD Alstätte 01,05554004,POINT (357094.000 5777294.000)
73897,100029930,2020-08-06,1244,1.00000,Konzentration zu gering zur Bestimmung,<,LGD Sielhorst 01,05770044,POINT (470220.000 5814318.000)


## change the type of "datum_pn" to datetime

In [34]:
df_merg['datum_pn'] = pd.to_datetime(df_merg['datum_pn'], format = '%Y-%m-%d')

In [35]:
df_merg.dtypes

messstelle_id                    int64
datum_pn                datetime64[ns]
stoff_nr                         int64
messergebnis_c                 float64
messergebnis_hinweis            object
messergebnis_cm                 object
name                            object
gemeinde_id                     object
geometry                      geometry
dtype: object

## create the geometry column

In [36]:
gdf_merg = gpd.GeoDataFrame(df_merg, geometry= 'geometry', crs="EPSG:25832")

In [37]:
gdf_merg.dtypes

messstelle_id                    int64
datum_pn                datetime64[ns]
stoff_nr                         int64
messergebnis_c                 float64
messergebnis_hinweis            object
messergebnis_cm                 object
name                            object
gemeinde_id                     object
geometry                      geometry
dtype: object

## Path directory for output

In [38]:
file_fname = r"nitrat_local.gpkg"

file_pfname = data_in_dir + file_fname

## Create shape file

In [39]:
gdf_merg.to_file(file_pfname, layer='nitrat_local', driver="GPKG")

# Finish !
## Good Luck, You are doing well